In [ ]:
import utils
import ingest
import dotenv
import pandas as pd
from config import load_config as load, load_db_opts, load_polygon_opts
from scrap import _scrap_sp_500_constituents

In [ ]:
# Load local dotenv (because not dockerize for now - can be added to image or passed at docker run)
dotenv.load_dotenv(utils.ENV_DOCKER_FILE)
dotenv.load_dotenv(utils.ENV_SECRETS_FILE)
config = load(load_db_opts, load_polygon_opts)

In [ ]:
# get tickers
# get each tickers earning dates
# roll 5 days before earning dates values
# get minute graphs for  | 5 days roll | > 10%

In [ ]:
# get tickers
from scrap import _scrap_sp_500_constituents

sp500_constituents = _scrap_sp_500_constituents()

In [ ]:
# get each tickers earning dates
from scrap import _scrap_previous_earnings
all_earning_dates = pd.DataFrame()

for _, row in sp500_constituents.head(5).iterrows():
    symbol = row["symbol"]
    curr_df = _scrap_previous_earnings(symbol)
    all_earning_dates = pd.concat([all_earning_dates, curr_df])

# filter on past earning dates only
from datetime import datetime

earning_dates = all_earning_dates[all_earning_dates["earnings_date"] < datetime.now()]

In [28]:
print(earning_dates.columns)

Index(['symbol', 'company', 'earnings_date', 'eps_estimates', 'eps_reported',
       'surprise_percent'],
      dtype='object')


In [36]:
# get data for each tickers
from datetime import timedelta
from scrap import _scrap_daily_prices_for_earnings

all_prices_around_earnings = pd.DataFrame()

for _, row in earning_dates.iterrows():
    symbol = row["symbol"]
    earnings_date = row["earnings_date"]
    earnings_date_5_days_before = earnings_date - timedelta(days=5)

    curr_df = _scrap_daily_prices_for_earnings(symbol, earnings_date_5_days_before, earnings_date)
    
    all_prices_around_earnings = pd.concat([all_prices_around_earnings, curr_df])

print(all_prices_around_earnings.columns)

- ABT: No data found for this date range, symbol may be delisted
Index(['date', 'open', 'high', 'low', 'close', 'volume', 'dividends',
       'stock_splits', 'symbol'],
      dtype='object')


In [37]:
# join earning dates & prices around earnings
earnings_with_lagging_prices = pd.merge(all_prices_around_earnings, earning_dates, left_on=["symbol", "date"], right_on=["symbol", "earnings_date"], how="left")
print(earnings_with_lagging_prices.columns)

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'dividends',
       'stock_splits', 'symbol', 'company', 'earnings_date', 'eps_estimates',
       'eps_reported', 'surprise_percent'],
      dtype='object')


In [64]:
# transform dates to day dates
earning_dates["earnings_day"] = earning_dates["earnings_date"].dt.date.to_string()
all_prices_around_earnings["day"] = all_prices_around_earnings["date"].dt.date.to_string()

# merge earning dates & prices around earnings
earnings_with_lagging_prices = pd.merge(earning_dates, all_prices_around_earnings, left_on=["symbol", "earnings_day"], right_on=["symbol", "day"], how="right")
print(earnings_with_lagging_prices.columns)

AttributeError: Can only use .str accessor with string values!

In [58]:
print(all_prices_around_earnings.dtypes)

date            datetime64[ns]
open                   float64
high                   float64
low                    float64
close                  float64
volume                 float64
dividends              float64
stock_splits           float64
symbol                  object
day                     object
dtype: object


In [59]:
print(earning_dates.dtypes)

symbol                      object
company                     object
earnings_date       datetime64[ns]
eps_estimates              float64
eps_reported               float64
surprise_percent            object
earnings_day                object
dtype: object


In [63]:
print(earnings_with_lagging_prices)

       symbol        company       earnings_date  eps_estimates  eps_reported  \
0         MMM     3M Company 2023-07-25 02:00:00           1.72          2.17   
1         MMM     3M Company 2023-04-25 02:00:00           1.58          1.97   
2         MMM     3M Company 2023-01-24 01:00:00           2.36          2.28   
3         MMM     3M Company 2022-10-25 02:00:00           2.60          2.69   
4         MMM     3M Company 2022-07-26 02:00:00           2.42          2.48   
...       ...            ...                 ...            ...           ...   
147131   ABMD  Abiomed, Inc. 1998-05-28 00:00:00          -0.08         -0.09   
147132   ABMD  Abiomed, Inc. 1998-01-28 00:00:00          -0.05         -0.07   
147133   ABMD  Abiomed, Inc. 1995-05-16 00:00:00          -0.01           NaN   
147134   ABMD  Abiomed, Inc. 1995-01-24 00:00:00          -0.01           NaN   
147135   ABMD  Abiomed, Inc. 1994-10-26 00:00:00          -0.01           NaN   

       surprise_percent ear